<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/DenseNet121_with_Cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install tf-nightly

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import backend as K

# import cifar 100 data
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# create the base pre-trained model
base_model = DenseNet121(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 100 classes
predictions = Dense(100, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test), verbose=1,batch_size=128)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Epoch 1/10
391/391 [==============================] - 23s 60ms/step - loss: 3.2355 - accuracy: 0.2170 - val_loss: 2.7330 - val_accuracy: 0.3068
Epoch 2/10
391/391 [==============================] - 21s 54ms/step - loss: 2.6463 - accuracy: 0.3241 - val_loss: 2.5447 - val_accuracy: 0.3450
Epoch 3/10
391/391 [==============================] - 21s 53ms/step - loss: 2.4300 - accuracy: 0.3662 - val_loss: 2.4851 - val_accuracy: 0.3567
Epoch 4/10
391/391 [==============================] - 21s 53ms/step - loss: 2.2683 - accuracy: 0.3990 - val_loss: 2.4515 - val_accuracy: 0.3731
Epoch 5/10
391/391 [==============================] - 21s 54ms/step - loss: 2.1344 - accuracy: 0.4300 - val_loss: 2.3990 - val_accuracy: 0.3885
Epoch 6/10
391/391 [==============================] - 21s 54ms/step - loss: 2.0101 - accuracy: 0.4571 - val_loss: 2.4003 - val_accuracy: 0.3883
Epoch 7/10
391/391 [==============================] - 21s 53ms/

In [20]:
# Fine Tuning
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
#for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)

# we chose to freeze the first 423 layers and unfreeze the rest:
for layer in model.layers[:423]:
   layer.trainable = False
for layer in model.layers[423:]:
   layer.trainable = True

# we train our model again (this time fine-tuning 
model.fit(x_train,y_train,epochs=25, validation_data=(x_test,y_test), verbose=1,batch_size=64)

Epoch 1/25
782/782 [==============================] - 32s 41ms/step - loss: 1.8079 - accuracy: 0.4989 - val_loss: 2.5541 - val_accuracy: 0.3719
Epoch 2/25
782/782 [==============================] - 33s 42ms/step - loss: 1.6585 - accuracy: 0.5349 - val_loss: 2.6040 - val_accuracy: 0.3767
Epoch 3/25
782/782 [==============================] - 32s 42ms/step - loss: 1.5464 - accuracy: 0.5604 - val_loss: 2.6337 - val_accuracy: 0.3803
Epoch 4/25
782/782 [==============================] - 32s 41ms/step - loss: 1.4426 - accuracy: 0.5847 - val_loss: 2.6663 - val_accuracy: 0.3836
Epoch 5/25
782/782 [==============================] - 32s 41ms/step - loss: 1.3803 - accuracy: 0.6052 - val_loss: 2.6793 - val_accuracy: 0.3858
Epoch 6/25
782/782 [==============================] - 32s 41ms/step - loss: 1.3145 - accuracy: 0.6163 - val_loss: 2.7913 - val_accuracy: 0.3819
Epoch 7/25
782/782 [==============================] - 32s 41ms/step - loss: 1.2537 - accuracy: 0.6319 - val_loss: 2.8198 - val_accuracy:

KeyboardInterrupt: ignored

In [21]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d_8 (ZeroPadding2D (None, None, None, 3 0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_8[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________